In [1]:
import numpy as np

In [37]:
class HGSO:
    def __init__(self):
        self.l1 = 5e-2
        self.l2 = 100
        self.l3 = 1e-2
        self.maximum_optimization = 1
        self.t0 = 298.15
        self.K = 1
        self.epsilon=0.05
        self.c1 = 0.1
        self.c2 = 0.2
        self.compiled = 0
        self.population = None
        self.fitnesses = None
        self.X_best = None
        self.Xi_best = None
        self.best_fitness = None
        
    def compile(self,obj,lb,ub,dim,n_population=100,n_cluster=5,alpha=1.0,beta=1.0,optimization="maximize"):
        if n_population%n_cluster != 0:
            raise Exception("Number of search agent in each cluster shoudl be same")
        self.dim = dim
        self.lb = lb
        self.ub = ub
        self.alpha = alpha
        self.beta = beta
        self.n_population = n_population
        self.n_cluster = n_cluster
        self.optimization = optimization
        self.comp_id = 0 if optimization == "maximize" else -1
        self.obj = obj
        self.compiled = 1
    
    def __get_fitness(self,X):
        if self.optimization == "maximize":
            return self.obj(X)
        elif self.optimization == "minimize":
            return 1/(self.obj(X)+0.002)
        
    def __initialize_population(self,dim,n_population,n_cluster):
        population = np.random.uniform(self.lb,self.ub,size=(self.n_population,self.dim))
        fitness = []
        for x in population:
            fitness.append(self.__get_fitness(x))
        return population,np.array(fitness)
    
    def __initialize_constant(self,n_population,n_cluster):
        h_j = (self.l1 * np.random.uniform(0,1,n_cluster)).reshape(-1,1)
        C_j = (self.l3 * np.random.uniform(0,1,n_cluster)).reshape(-1,1)
        n_j = n_population//n_cluster
        P_ij = self.l2 * np.random.uniform(0,1,size=(n_cluster,n_j))
        return h_j,C_j,P_ij
    
    def __get_cluster_best(self,population_fitness,n_cluster):
        n_element = self.n_population//n_cluster
        cluster_best = []
        
        for i in range(n_cluster):
            li = i*n_element
            ui = li + n_element
            best = np.argsort(population_fitness[li:ui])[-1]
            cluster_best.append(i*n_element+best)
        return np.array(cluster_best)
    
    def __get_population_best(self,population_fitness):
        return self.__get_cluster_best(population_fitness,1)
        
    def __update_h_j(self,max_iter,t):
        T = t/max_iter
        h_j = self.h_j * np.exp(-self.C_j*((1.0/T)-(1.0/self.t0)))
        
        return h_j
    
    def __update_S_ij(self,h_j,P_ij):
        S_ij = self.K * h_j * P_ij
        
        return S_ij
    
    def __update_worst_agent(self):
        #Getting the value of N_w
        N_w = int(self.n_population * (np.random.uniform(0,self.c2-self.c1)+self.c1))
        # Getting the index of worst agent in the population
        
        worst_pop_index = np.argsort(self.fitnesses)[:N_w]
        #print(f"N_w: {N_w}")
        # Updating the worst agent in the population
        self.population[worst_pop_index] = np.random.uniform(self.lb,self.ub,size=(N_w,self.dim))
        # Updating the fitnesses of newly assigned agent
        for i in worst_pop_index:
            self.fitnesses[i] = self.__get_fitness(self.population[i])
    
    def __update_global_best(self):
        best_id = self.__get_population_best(self.fitnesses)
        #print(best_id)
        if self.best_fitness == None or self.best_fitness < self.fitnesses[best_id]:
            self.X_best = self.population[best_id]
            self.best_fitness = self.fitnesses[best_id]
        
    def __update_population(self):
        # Setting the F value according to a random number to make the update diverse
        F = []
        for i  in range(self.n_population):
            f = -1 if np.random.uniform() < 0.5 else 1
            F.append(f)
        F = np.array(F).reshape(-1,1)
        #print(F.shape)
        #print("Hello World")
        
        # r is a random value
        r = np.random.uniform(0,1,size=(self.n_population,1))
        
        # Getting the number of population in a clusters
        n_cluster_population = self.n_population//self.n_cluster
        
        #Getting the best search agent in cluster and best agent in the whole population
        X_best = self.X_best
        Xi_best = self.population[self.Xi_best].repeat(n_cluster_population,axis=0)
        
        # Getting the value of constant  gamma
        epsilon = 0.05
        best_fitnesses = self.fitnesses[self.Xi_best].repeat(n_cluster_population)

        gamma = self.beta * np.exp(-(best_fitnesses+self.epsilon)/(self.fitnesses+self.epsilon))
        gamma = gamma.reshape(-1,1)
        updated_population = self.population * F * r * gamma * (Xi_best-self.population) + F * r * self.alpha * (self.S_ij.reshape(-1,1) * X_best + self.population)
        updated_population = np.clip(updated_population,self.lb,self.ub)
        #print(gamma.shape)
        fitnesses = []
        for i in updated_population:
            fitnesses.append(self.__get_fitness(i))
        return updated_population,np.array(fitnesses)
    
    def train(self,max_iter):
        # Checking if compiled
        if self.compiled == 0:
            raise Exception("You have to compile first.")
        # Initializing the population and cluster, calculating the fitness of the populations
        self.population,self.fitnesses= self.__initialize_population(self.dim,self.n_population,self.n_cluster)
        
        # Intializing the Necessary constant
        self.h_j,self.C_j,self.P_ij = self.__initialize_constant(self.n_population,self.n_cluster)
        # Getting the fitness and getting the best gasses in the clusters and in the swarm
        
        # Getting the index of best search agent in population and clusters
        #self.X_best = self.__get_population_best(self.fitnesses)
        #self.Xi_best = self.__get_cluster_best(self.fitnesses,self.n_cluster)
        
        for i in range(max_iter):
            
            # Evaluating the current populations using fitnesses
            self.__update_global_best()
            
            self.Xi_best = self.__get_cluster_best(self.fitnesses,self.n_cluster)
            
            # Updating the Henry Gas Constant for each cluster
            self.h_j = self.__update_h_j(max_iter,i+1)

            # Updating the solubility of the gas
            self.S_ij = self.__update_S_ij(self.h_j,self.P_ij)
            self.population,self.fitnesses = self.__update_population()
            
            # Updating the position of worst agent in the population
            self.__update_worst_agent()
            if (i+1) % 10 == 0:
                print(f"Iter {i+1}:\t Best Value: {self.obj(self.X_best[0])}")

In [38]:
hgso = HGSO()

In [39]:
def Sphere_func(x):
    fitness = 0.0
    for i in range(len(x)):
        fitness += (x[i]*x[i])
    return fitness

dim = 30
lb = np.array([-10.0],dtype='f').repeat(dim)
ub = np.array([10.0],dtype='f').repeat(dim)
max_iter = 100

In [40]:
hgso.compile(Sphere_func,lb,ub,dim,optimization="minimize")

In [41]:
hgso.train(100)

Iter 10:	 Best Value: 2.5589016713451498e-15
Iter 20:	 Best Value: 6.367586790841844e-20
Iter 30:	 Best Value: 6.367586790841844e-20
Iter 40:	 Best Value: 6.367586790841844e-20
Iter 50:	 Best Value: 6.367586790841844e-20
Iter 60:	 Best Value: 6.367586790841844e-20
Iter 70:	 Best Value: 6.367586790841844e-20
Iter 80:	 Best Value: 6.367586790841844e-20
Iter 90:	 Best Value: 6.367586790841844e-20
Iter 100:	 Best Value: 6.367586790841844e-20


In [ ]:
sphere_func(x):
    X = [x1,x2,x3,x4]
    return x1**2+x2**2+x3**2+x4**2

In [3]:

class Sphere():
    def __init__(self,dim,opt_value):
        self.dim=dim
        self.opt_value = opt_value
        self.optimization = 'minimize'
    
    def get_algorithm(self):
        def sphere_func(x):
            res=0
            for i in x:
                res+=i**2
            return res
        return sphere_func
    
sphere = Sphere(3,0)
print(sphere.get_algorithm())

<function Sphere.get_algorithm.<locals>.sphere_func at 0x7f2354464820>


In [9]:
from benchmarking.benchmark import *

In [4]:
ac = Ackley()

In [5]:
ac.get_algorithm()

<function benchmarking.benchmark.Ackley.get_algorithm.<locals>.ackley_func(x)>

In [6]:
ac.dim

30

In [7]:
ac.range

(-32, 32)